## 关联挖掘

在数据集中搜索频繁出现的项目。

在频繁挖掘中，通常会发现事务性和关系性数据库中的项目集之间有趣的关联和相关性。简而言之，`频繁挖掘显示哪些项目在交易或关系中一起出现`。

比如，如果有两件物品X和Y经常被购买，那么最好把它们放在一起，或者在购买另一件物品时提供一些折扣优惠，这可以增加销售额。例如，如果顾客购买了牛奶和面包，他/她也会购买黄油。所以关联规则是['牛奶]^['面包']=>['黄油']。如果他/她买了牛奶和面包，卖家可以建议客户购买黄油。

定义：

1. Support 
   
   说明了规则的有用性和确定性，比如5%的支持率意味着数据中总共有5%的交易遵循该规则

   A关联B的support = A和B同时出现的次数

   ![20220719161656](https://cdn.jsdelivr.net/gh/xihuishawpy/PicBad@main/blogs/pictures/20220719161656.png)

2. Confidence 
   
   比如，60%的置信度意味着，有60%把握可推断，购买牛奶和面包的顾客也会购买黄油

   ![20220719161935](https://cdn.jsdelivr.net/gh/xihuishawpy/PicBad@main/blogs/pictures/20220719161935.png)

**如果一个规则同时满足最小支持度和最小置信度，它就是一个强规则。**

### 频繁项挖掘demo

#### 1. 发现频繁项目集

频繁项集合：

![20220719164009](https://cdn.jsdelivr.net/gh/xihuishawpy/PicBad@main/blogs/pictures/20220719164009.png)

1. 集合项目数为1的候选集（删除支持度计数小于2的项，以下过程重复）
   
   ![20220719164125](https://cdn.jsdelivr.net/gh/xihuishawpy/PicBad@main/blogs/pictures/20220719164125.png)

   比如，l1在初始频繁项集合中出现次数为6

2. 集合项目数为2的候选集
   
   ![20220719164500](https://cdn.jsdelivr.net/gh/xihuishawpy/PicBad@main/blogs/pictures/20220719164500.png)

   检查一个项目集的所有子集是否频繁，如果不频繁就删除该项目集（例如{I1, I2}的子集是{I1}，{I2}，它们是频繁的）


   ![20220719164736](https://cdn.jsdelivr.net/gh/xihuishawpy/PicBad@main/blogs/pictures/20220719164736.png)

   删除支持度计数小于2的项

3. 集合项目数为3的候选集

   ![20220719164905](https://cdn.jsdelivr.net/gh/xihuishawpy/PicBad@main/blogs/pictures/20220719164905.png)

   这里{I1, I2, I3}的子集是{I1, I2},{I2, I3},{I1, I3}是频繁的。对于{I2, I3, I4}，子集{I3, I4}不是频繁出现的，所以将其删除

4. 集合项目数为4的候选集

   这里的项目集是{I1, I2, I3, I5}，所以它的子集包含{I1, I3, I5}，这不是频繁的，所以C4中没有项目集

总结：每一类项目集都要满足2个条件，`支持度计数满足最小计数，并且其子集需要是频繁的（满足子集的支持度计数至少是最小计数）`


### 2. 计算频繁项目集的规则置信度

以项目集 {I1, I2, I3} 举例：

如果最小置信度是50%，那么前3条规则就可以被认为是强关联规则。

![20220719165504](https://cdn.jsdelivr.net/gh/xihuishawpy/PicBad@main/blogs/pictures/20220719165504.png)

## Apriori算法示例

原始数据：

![20220719170544](https://cdn.jsdelivr.net/gh/xihuishawpy/PicBad@main/blogs/pictures/20220719170544.png)

In [ ]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules


# 将交易转换为用户-项目矩阵，这样每一列都是一个项目。
# 如果一个项目属于一个交易，则矩阵中的每个值为1，如果一个项目不属于一个交易，则为0。
hot_encoded_df = df.groupby(['Transaction', 'Item'])['Item'].count().unstack().fillna(0)

![20220719170618](https://cdn.jsdelivr.net/gh/xihuishawpy/PicBad@main/blogs/pictures/20220719170618.png)

In [ ]:
# 预处理
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1
    
hot_encoded_df = hot_encoded_df.applymap(encode_units)

In [ ]:
# 支持阈值=0.01，代表至少发生了94笔交易
frequent_itemsets = apriori(hot_encoded_df, min_support = 0.01, use_colnames = True)



![20220719170814](https://cdn.jsdelivr.net/gh/xihuishawpy/PicBad@main/blogs/pictures/20220719170814.png)

In [ ]:
# 以提升度和置信度为指标找到强关联规则

rules = association_rules(frequent_itemsets, metric = "lift", min_threshold = 1)
rules.sort_values('confidence', ascending = False, inplace = True)
rules[['antecedents','consequents','support','confidence','lift']].head(5)

![20220719171151](https://cdn.jsdelivr.net/gh/xihuishawpy/PicBad@main/blogs/pictures/20220719171151.png)

比如，对于'Toast-->Coffee'：
- Confidence ： 有2.37%的交易同时包含吐司和咖啡
- Lift ：大于1的提升度意味着产品A和B更有可能被一起购买，这里的提升度为1.47，意味着顾客一起购买吐司和咖啡的可能性是单独购买咖啡的1.47倍

注意：得到的结果不一定具有因果关系，但能表明item之间`有强烈的共现关系`。